## Отчет по лабороторной работе №2

Spider, соответствующий условию, находится в репозитории в папке [scrapy_wiki](scrapy_wiki). В качетстве стартовых использовались 4 страницы:
    1. https://en.wikipedia.org/wiki/Wiki
    2. https://en.wikipedia.org/wiki/Facebook'
    3. https://en.wikipedia.org/wiki/Belarus
    4. https://en.wikipedia.org/wiki/Philosophy
    
Spider работал пока не будет скачано около 16 000 страниц. Результаты работы сохранялись в формате jsonlines в файл [scrapy_wiki\scrapy_results\wiki_links.json](scrapy_wiki\scrapy_results\wiki_links.json).

Далее приведен листинг построения графа по данным из файла, запуска на нём алгоритма PageRank с разными значениями параметра alpha, алгоритма HITS.

In [1]:
import json
from operator import itemgetter

import networkx as nx


def load_graph(file, filter_not_loaded_nodes=True):
    graph = nx.DiGraph()

    url_to_info = dict()

    with open(file) as input_file:
        for line in input_file.readlines():
            node_info = json.loads(line)
            graph.add_node(node_info['url'])

            url_to_info[node_info['url']] = (node_info['title'], node_info['snippet'])

    with open(file) as input_file:
        for line in input_file.readlines():
            node_info = json.loads(line)
            node_url = node_info['url']
            for outlink in node_info['outlinks']:
                if not filter_not_loaded_nodes or outlink in url_to_info:
                    graph.add_edge(node_url, outlink)

    return graph, url_to_info


def analyze(G, url_to_info):
    print("Nodes count: {}".format(G.number_of_nodes()))
    print("Edges count: {}\n".format(G.number_of_edges()))

    def print_page(page_url, value):
        if page_url in url_to_info:
            page_attrs = url_to_info[page_url]
            print("\t{} {}\n\t{}\n\t{}\n".format(page_attrs[0], value, page_url, page_attrs[1]))
        else:
            print("\t{} {}\n\t{}\n\t{}\n".format('<No Title>', value, page_url, '<No snippet>'))

    def print_top10(ranks):
        graph_page_rank_sorted = sorted(list(ranks.items()), key=itemgetter(1), reverse=True)

        for page in graph_page_rank_sorted[:10]:
            print_page(page[0], page[1])

    alphas = [0.3, 0.5, 0.85, 0.95]

    for alpha in alphas:
        print("Top 10 by PageRank for alpha = {}\n".format(alpha))
        print_top10(nx.pagerank(G, alpha=alpha))

    hubs, authorities = nx.hits(G, max_iter=500)
    average = {url: (value + authorities[url]) / 2 for url, value in hubs.items()}
    print("Top 10 HITS: hubs\n")
    print_top10(hubs)
    print("Top 10 HITS: authorities\n")
    print_top10(authorities)
    print("Top 10 HITS: average\n")
    print_top10(average)

Анализ графа проводился для двух случаев:
    1. При построении графа в качестве вершин использовались только скачанные страницы и, соответственно, учитывались только ребра которые есть между ними
    2. При построении графа в качестве вершин использовались все обнаруженные страницы, в том числе те, которые ещё не были скачаны

Результаты для первого случая:

In [2]:
G1, url_to_info1 = load_graph('scrapy_wiki/scrapy_results/wiki_links.json')
analyze(G1, url_to_info1)

Nodes count: 15993
Edges count: 33735

Top 10 by PageRank for alpha = 0.3

	Greek language 0.0014838931475941448
	https://en.wikipedia.org/wiki/Greek_language
	Greek (Modern Greek: ελληνικά [eliniˈka], elliniká, "Greek", ελληνική γλώσσα [eliniˈci ˈɣlosa] ( listen), ellinikí glóssa, "Greek language") is an independent branch of the Indo-European family of languages, native to Greece and other parts of the Eastern...

	DMOZ 0.0013840697839551885
	https://en.wikipedia.org/wiki/DMOZ
	DMOZ (from directory.mozilla.org, an earlier domain name) was a multilingual open-content directory of World Wide Web links. The site and community who maintained it were also known as the Open Directory Project (ODP). It was owned by AOL but constructed ...

	Internet 0.0012541832810415024
	https://en.wikipedia.org/wiki/Internet
	The Internet is the global system of interconnected computer networks that use the Internet protocol suite (TCP/IP) to link devices worldwide. It is a network of networks that consis

Для даного случая можно отметить следующие факты:
* Топ алгоритма PageRank сильно зависит от выбора стартовых страниц для Spider'а
* С увеличением alpha разрыв в значениях PageRank между рядом стоящими в топе страницами увеличивается (лидеры становятся более явными), и соответственно топ PageRank не так сильно меняется
* Топ HITS hubs содержит в первую очередь страницы, на которых есть большое количество ссылок на другие страницы
* Топы HITS authorities и HITS average во много повторяют топ PageRank

Результаты для второго случая:

In [3]:
G2, url_to_info2 = load_graph('scrapy_wiki/scrapy_results/wiki_links.json', False)
analyze(G2, url_to_info2)

Nodes count: 455711
Edges count: 939672

Top 10 by PageRank for alpha = 0.3

	<No Title> 3.504054205562873e-05
	https://en.wikipedia.org/wiki/United_States
	<No snippet>

	<No Title> 1.8707605998534233e-05
	https://en.wikipedia.org/wiki/United_Kingdom
	<No snippet>

	<No Title> 1.2560938920612364e-05
	https://en.wikipedia.org/wiki/IMDb
	<No snippet>

	<No Title> 1.251901286570417e-05
	https://en.wikipedia.org/wiki/World_War_II
	<No snippet>

	<No Title> 1.1567140621825778e-05
	https://en.wikipedia.org/wiki/New_York_City
	<No snippet>

	<No Title> 1.1367872640139448e-05
	https://en.wikipedia.org/wiki/Australia
	<No snippet>

	<No Title> 1.108417938518168e-05
	https://en.wikipedia.org/wiki/Canada
	<No snippet>

	<No Title> 1.0850949108296007e-05
	https://en.wikipedia.org/wiki/France
	<No snippet>

	<No Title> 1.0848592666827382e-05
	https://en.wikipedia.org/wiki/Germany
	<No snippet>

	<No Title> 9.877824455822137e-06
	https://en.wikipedia.org/wiki/London
	<No snippet>

Top 10 by PageRan

Для даного случая можно отметить следующие факты:
* Выдача алгоритма PageRank в меньшей степени зависит от выбора стартовых страниц для Spider'а, потому что ссылки на действительно популярные страницы уже были обнаружены
* С увеличением alpha топ PageRank не меняется, это обусловлено тем что данных о ссылках для большинства страниц нет, хотя данный топ уже больше похож на тот, который получился бы при запуске PageRank на настоящем графе, соответствующем домену en.wikipedia.org
* Топ HITS hubs содержит в первую очередь страницы, на которых есть большое количество ссылок на другие страницы (просто по сравнению с предыдущим случаем этот список стал другим, так как мы перестали отфильтровывать большое кол-во ссылок)
* Топ HITS authorities во много повторяет топ PageRank, что и следовало ожидать
* Топ HITS average, кажется, не имеет большого смысла 